<a href="https://colab.research.google.com/github/Luizcarlosqueiroz/Otimizacao_por_Enxame_de_Particulas/blob/main/Otimiza%C3%A7%C3%A3o_por_Enxame_de_Part%C3%ADculas_Sphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importar as Bibliotecas Básicas

In [3]:
import random
import math
import numpy as np 

## Função Objetivo (Sphere)

$$f(x_1 \cdots x_n) = \sum_{i=1}^n x_i^2$$

$$-5.12 \leq x_i \leq 5.12$$

$$\text{minimum at }f(0, \cdots, 0) = 0$$ 

In [10]:
#https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/sphere.html

def funcao_objetivo(x):
  z = np.sum(np.square(x))

  return z

### Posição Inicial e Atualização

In [4]:
def initPosition(Np, Nd, xMin, xMax):
  return xMin + np.random.rand(Np, Nd)*(xMax-xMin)


In [5]:
def updatePosition (R, V, Np, Nd, xMin, xMax):

  R += V

  # Caso a particula fuja do domínio de busca desejado
  for particle in range(Np):
    for dimension in range(Nd):
      if R[particle][dimension] > xMax:
        R[particle][dimension] = xMax
      if R[particle][dimension] < xMin:
        R[particle][dimension] = xMin
        
  return R

### Velocidade e Atualização

In [6]:
def initVelocity(Np, Nd, vMin, vMax):
    V = vMin + np.random.rand(Np, Nd)*(vMax-vMin)
    return V

In [7]:
def updateVelocity(R, V, Np, Nd, w, c1, c2, vMin, vMax, chi, pBestPos, gBestPos):
    
    r1 = np.random.rand()
    r2 = np.random.rand()

    for particle in range(Np):
        V[particle, :] = chi * (w * V[particle, :] + r1*c1*(pBestPos[particle, :] - R[particle, :]) + r2*c2*(gBestPos - R[particle, :]))

        for dimension in range(Nd):
            if V[particle][dimension] > vMax: 
                V[particle][dimension] = vMax
            if V[particle][dimension] < vMin: 
                V[particle][dimension] = vMin

    return V

In [8]:
def updateFitness(R, Np, gBestValue, gBestPos, pBestValue, pBestPos):

    for particle in range(Np):
        M = funcao_objetivo(R[particle, :])

        if M < gBestValue:
            gBestValue = M
            gBestPos = R[particle, :]

        if M < pBestValue[particle]:
            pBestValue[particle] = M
            pBestPos[particle, :] = R[particle, :]

    return gBestValue, gBestPos, pBestValue, pBestPos

## Código Final

In [11]:
lista_de_gbests = []

for _ in range(30):

  #Número de particulas, Número de dimensões, Número de Iterações
  Np, Nd, Nt = 20, 2, 500
  #Pesos para atualizar a velocidade (seguindo o padrão)
  c1, c2 = 2.05 , 2.05
  #Definição do min e max do coeficiente de inércia
  wMin, wMax = 0.4, 0.9

  #Espaço (de acordo com os limites dados pela função de Rastring)
  xMin, xMax = -5.12 , 5.12
  #Velocidades (imaginando que as particulas podem se mover menos, dado um espaço amostral menor)
  vMin, vMax = 0.15*xMin , 0.15*xMax

  #Visto que o problema é de minimização, inicializando os best's com valor muito altos
  gBestValue = float("inf")
  pBestValue = [float("inf")] * Np

  pBestPos = np.zeros((Np, Nd))
  gBestPos = np.zeros(Nd)

  #Calculando o fator de Clerck
  #Considerando phi > 4, o chi será de 0.7, fazendo com que a V esteja sempre diminuindo
  phi = c1 + c2
  chi = 2.0/abs(2.0-phi-np.sqrt(pow(phi, 2)-4*phi))

  #Inicializando as particulas e velocidade
  R = initPosition(Np, Nd, xMin, xMax)
  V = initVelocity(Np, Nd, vMin, vMax)

  history = []

  for j in range (0, Nt):

    R = updatePosition(R, V, Np, Nd, xMin, xMax)

    gBestValue, gBestPos, pBestValue, pBestPos = updateFitness(R, Np, gBestValue, gBestPos, pBestValue, pBestPos)
    history.append(gBestValue)

    w = wMax - ((wMax-wMin)/Nt)*j
    V = updateVelocity(R, V, Np, Nd, w, c1, c2, vMin, vMax, chi, pBestPos, gBestPos)

  lista_de_gbests.append(gBestValue)

In [12]:
lista_de_gbests

[2.00284328707823e-104,
 2.1045508830327405e-112,
 1.3683356748176795e-104,
 8.396526588105391e-101,
 4.440187863969121e-107,
 2.6221304806199447e-103,
 3.9963264748761457e-106,
 8.717852446122819e-106,
 6.517054189122477e-105,
 4.461461748846821e-108,
 7.1362894153960506e-99,
 3.979666700800981e-104,
 2.019751663012875e-105,
 1.881773592075804e-71,
 2.951423552377135e-105,
 3.6485805477095182e-99,
 3.668784628430352e-107,
 1.7230330858728014e-103,
 4.065870672496155e-96,
 3.258238490495742e-109,
 8.537136457476076e-92,
 2.687602639764024e-98,
 2.073670832771391e-102,
 2.1852692893661194e-65,
 6.554809580020007e-111,
 2.8524951418286423e-102,
 2.7889598917716e-102,
 3.7356460077528495e-101,
 3.765422745469166e-105,
 7.700986500878657e-60]